## Processing T1w images to estimate Gray Matter Density


In [11]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber, FreeSurferSource
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm.preprocess import VBMSegment, Segment
from nipype.interfaces.ants import Atropos, Registration, ApplyTransforms, N4BiasFieldCorrection
from nipype.interfaces.fsl import ApplyMask, BET
from nipype.interfaces.freesurfer import FSCommand
from pandas import DataFrame, Series, read_csv

# FSL setup- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study specific variables
study_home = '/moochie/Cat/Aggregate_anats/GMD_ML'

subject_info = read_csv(study_home + '/doc/subject_info.csv', index_col=None)
subjects_list = subject_info['freesurferID'].tolist()
#subjects_list = ['101']

raw_data = study_home + '/raw'
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'

sample_template = study_home + '/templates/lcbd_template_1mm.nii.gz'
sample_template_brain = study_home + '/templates/lcbd_template_1mm_brain.nii.gz'
sample_template_mask = study_home + '/templates/lcbd_template_1mm_mask.nii.gz'

# tissue priors
csf_prior = study_home + '/templates/tissue_priors/tissue0.nii.gz'
wm_prior = study_home + '/templates/tissue_priors/tissue3.nii.gz'
gm_prior = study_home + '/templates/tissue_priors/gm_tissue.nii.gz'

#freesurfer setup
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

In [12]:
# Data handling nodes
infosource = Node(IdentityInterface(fields=['subjid']), 
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

fs_source = Node(FreeSurferSource(subjects_dir=fs_dir), 
                 name='fs_source')

substitutions = [('_subjid_', '')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=output_dir,
                         container=output_dir), 
                name='datasink')

### Bias-correct and segment subject images

In [15]:
# N4 bias correction
biascorrect = Node(N4BiasFieldCorrection(save_bias=True),name='biascorrect')

# Register to template brain
reg_temp1 = Node(Registration(fixed_image=sample_template,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=4,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Rigid', 'Affine'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp1')

# Skullstripping
#skullstrip = Node(ApplyMask(mask_file=sample_template_mask),name='skullstrip')
skullstrip = Node(BET(),name='skullstrip')

# Register to skullstripped brain
reg_temp2 = Node(Registration(fixed_image=sample_template_brain,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=4,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Affine', 'SyN'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp2')

# Atropos segmentation
segment = Node(Atropos(initialization='PriorProbabilityImages',
                       prior_probability_images=[wm_prior, gm_prior, csf_prior],
                       number_of_tissue_classes=3, 
                       prior_weighting = 0.5,
                       prior_probability_threshold = 0.5,
                       num_threads =4,
                       mask_image=sample_template_mask, 
                       save_posteriors=True, n_iterations=10),name='segment')

In [16]:
segproc = Workflow(name='segproc')
segproc.connect([(infosource, fs_source,[('subjid','subject_id')]),
                 (fs_source,biascorrect,[('T1','input_image')]),
                 (biascorrect,reg_temp1,[('output_image','moving_image')]),
                 (reg_temp1, skullstrip, [('warped_image','in_file')]),
                 (skullstrip, reg_temp2, [('out_file','moving_image')]), 
                 (reg_temp2, segment, [('warped_image','intensity_images')]),
                 
                 (reg_temp1, datasink, [('composite_transform','reg1_composite_transform'), 
                                        ('inverse_composite_transform','reg1_inverse_composite_transform'),
                                        ('warped_image','reg1_warped_anat')]),
                 (reg_temp2, datasink, [('composite_transform','reg2_composite_transform'),
                                        ('inverse_composite_transform','reg2_inverse_composite_transform'),
                                        ('warped_image','reg2_warped_anat')]),
                 (segment, datasink, [('classified_image','hard_tissue_files'),('posteriors','soft_tissue_files')])
                 
                ])
segproc.base_dir = workflow_dir
segproc.write_graph(graph2use='flat')
segproc.run('MultiProc', plugin_args={'n_procs': 28,'memory_gb':30})

190327-08:45:19,871 workflow INFO:
	 Generated workflow graph: /moochie/Cat/Aggregate_anats/GMD_ML/workflows/segproc/graph.png (graph2use=flat, simple_form=True).
190327-08:45:20,119 workflow INFO:
	 Workflow segproc settings: ['check', 'execution', 'logging', 'monitoring']
190327-08:45:21,662 workflow INFO:
	 Running in parallel.
190327-08:45:21,842 workflow INFO:
	 [MultiProc] Running 0 tasks, and 170 jobs ready. Free memory (GB): 30.00/30.00, Free processors: 28/28.
190327-08:45:21,977 workflow INFO:
	 [Node] Setting-up "segproc.fs_source" in "/moochie/Cat/Aggregate_anats/GMD_ML/workflows/segproc/_subjid_2027/fs_source".190327-08:45:21,979 workflow INFO:
	 [Node] Setting-up "segproc.fs_source" in "/moochie/Cat/Aggregate_anats/GMD_ML/workflows/segproc/_subjid_1005/fs_source".190327-08:45:21,980 workflow INFO:
	 [Node] Setting-up "segproc.fs_source" in "/moochie/Cat/Aggregate_anats/GMD_ML/workflows/segproc/_subjid_C1027/fs_source".190327-08:45:21,981 workflow INFO:
	 [Node] Setting-up